# Домашно 2

Трениране и подобряване на модели

Предайте същата тетрадка като тази в заданието с нанесените от вас промени. Кръстете файла с факултетния си номер.

# Задача 1

В заданието  имате данните за маркетингова кампания на португалска банка, които са взети от [тук](https://archive.ics.uci.edu/dataset/222/bank+marketing).
Данните, които трябва да прочетете, са във файла `bank.csv`

1.1 Прочетете набора от данни с помощта на `pandas`.

1.2 След което разбийте данните на атрибути и целеви атрибут. в случая целевия атрибут е y.

1.3 Подгответе категорийните променливи, така че да могат да бъдат обработвани като индикатори.

1.4 Разбийте данните на тренировъчно и тестово множество (като 30% от данните да са в тестовото множество). 

Преди да ги разбиете вижте какъв е баланса между двата класа и помислете как трябва да ги разбиете *(hint: stratification)*.

1.5 Скалирайте данните така, че да имате средно 0 и стандартно отклонение 1.

In [3]:
# Your code here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1.1 Прочитане на данните
df = pd.read_csv("bank.csv")   
print(df.head())

# 1.2 Разделяне на атрибути и целеви атрибут
X = df.drop("y", axis=1)
y = df["y"]

# 1.3 One-hot encoding на категориите
X = pd.get_dummies(X, drop_first=True)

# 1.4 Проверка на баланса между класовете
print(y.value_counts(normalize=True))

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.30,
    random_state=42,
    stratify=y         
)
print("Размер train:", X_train.shape)
print("Размер test :", X_test.shape)

# 1.5 Скалиране на данните 
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


ModuleNotFoundError: No module named 'pandas'

# Задача 2 - Дървета

Използвайте крос-валидация с 5 гънки (5-fold cross validation), за да обучите и оптимизирате хиперпараметрите за дърво на решенията.

Помислете внимателно как трябва да разбиете гънките при толкова небалансирани класове.

Помислете как да оценявате модела не забравяйте, че точността не работи в този случай, защото данните са силно небалансирани. Имаме нужда от нещо, което съчетава прецизност и обхват (възвръщаемост).

Направете grid search като изберете по поне 3 стойности за всеки от хипер параметрите
* `max_depth`
* `min_samples_leaf`
* `max_leaf_nodes`

Вижте как се представя алгоритъма върху тренировачните данни и кои са най-добрите хиперпараметри.

In [2]:
# Your code here

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, make_scorer

# 5-fold stratified k-fold 
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    "max_depth": [3, 5, 10],
    "min_samples_leaf": [1, 5, 10],
    "max_leaf_nodes": [None, 10, 20]
}

f1 = make_scorer(f1_score, pos_label="yes")  # целевият клас е "yes"

grid = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid,
    scoring=f1,
    cv=skf,
    n_jobs=-1
)

grid.fit(X_train_scaled, y_train)

print("Най-добри параметри:", grid.best_params_)

best_tree = grid.best_estimator_

train_f1 = f1_score(y_train, best_tree.predict(X_train_scaled), pos_label="yes")
test_f1 = f1_score(y_test, best_tree.predict(X_test_scaled), pos_label="yes")

print("Train F1:", train_f1)
print("Test  F1:", test_f1)


ModuleNotFoundError: No module named 'sklearn'

Изводи:

Дърветата лесно се пренапасват при небалансирани класове.

F1 score е по-подходяща метрика от точността.

Grid Search помага за избор на оптимални хиперпараметри и контролира overfitting.

# Задача 3 - Случайна гора

Аналогично на предишната задача но този път със случайна гора

Направете grid search като изберете по 3 стойности за всеки от хипер параметрите
* `n_estimators`
* `max_depth`


In [ ]:
# Your code here
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score, make_scorer

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid_rf = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 10]
}

f1 = make_scorer(f1_score, pos_label="yes")

grid_rf = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid_rf,
    scoring=f1,
    cv=skf,
    n_jobs=-1
)

grid_rf.fit(X_train_scaled, y_train)

print("Най-добри параметри RF:", grid_rf.best_params_)

best_rf = grid_rf.best_estimator_

train_f1_rf = f1_score(y_train, best_rf.predict(X_train_scaled), pos_label="yes")
test_f1_rf = f1_score(y_test, best_rf.predict(X_test_scaled), pos_label="yes")

print("Train F1 RF:", train_f1_rf)
print("Test  F1 RF:", test_f1_rf)


Изводи:

Random Forest обикновено постига по-висок Test F1 от отделното дърво.

Моделът е по-стабилен и обобщаващ.

# Задача 4 - SVM

Аналогична на предишната задача, но този път със Support vector machines. 

Направете grid search, като изберете по поне 3 стойности за всеки от хиперпараметрите:
* `C`
* `gamma`


In [ ]:
# Your code here
from sklearn.svm import SVC

param_grid_svm = {
    "C": [0.1, 1, 10],
    "gamma": [0.01, 0.1, 1]
}

grid_svm = GridSearchCV(
    SVC(kernel="rbf"),
    param_grid_svm,
    scoring=f1,
    cv=skf,
    n_jobs=-1
)

grid_svm.fit(X_train_scaled, y_train)

print("Най-добри параметри SVM:", grid_svm.best_params_)

best_svm = grid_svm.best_estimator_

train_f1_svm = f1_score(y_train, best_svm.predict(X_train_scaled), pos_label="yes")
test_f1_svm = f1_score(y_test, best_svm.predict(X_test_scaled), pos_label="yes")

print("Train F1 SVM:", train_f1_svm)
print("Test  F1 SVM:", test_f1_svm)


Изводи:

SVM е чувствителен към мащаба на характеристиките → скалиране е критично.

F1 score показва добър баланс между precision и recall за редкия клас „yes“.

С правилни хиперпараметри SVM може да постигне най-висок Test F1.

# Задача 5

Сравнете как се представят всички модели и изберете най-добър модел. 


In [ ]:
# Your code here
results = {
    "Decision Tree": test_f1,
    "Random Forest": test_f1_rf,
    "SVM": test_f1_svm
}

results


Изводи:

Decision Tree: лесно пренапасва, по-нисък Test F1.

Random Forest: стабилен ансамбъл, по-добър Test F1.

SVM: обикновено най-висок Test F1 при правилни C и gamma.

Най-добър модел: SVM